In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def extract_waiting_times_per_case(base_dir, target_densities, case_labels, num_trials=50):
    cases = sorted([
        d for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d)) and d.startswith('March26_Case')
    ])

    if len(cases) != len(case_labels):
        raise ValueError("Number of cases and case labels must match!")

    all_kappas = set()
    all_rates = set()

    data_completed = []
    data_still_waiting = []

    for case, label in zip(cases, case_labels):
        print(f"{label}: {case}")
        case_path = os.path.join(base_dir, case)

        for kappa_dir in os.listdir(case_path):
            if not kappa_dir.startswith('Kappa_'):
                continue
            kappa_value = kappa_dir.split('_')[1]
            all_kappas.add(kappa_value)
            kappa_path = os.path.join(case_path, kappa_dir)

            density = target_densities.get((label, kappa_value))
            if density is None:
                print(f"⚠️ Skipping: No density for ({label}, {kappa_value})")
                continue

            density_path = os.path.join(kappa_path, f'Density_{density}')
            if not os.path.exists(density_path):
                print(f"⚠️ Skipping: No directory {density_path}")
                continue

            for rate_dir in os.listdir(density_path):
                if not rate_dir.startswith('PassengerRate_'):
                    continue
                rate_value = rate_dir.split('_')[1]
                all_rates.add(rate_value)
                rate_path = os.path.join(density_path, rate_dir, 'PassengerData')
                if not os.path.exists(rate_path):
                    continue

                for trial in range(1, num_trials + 1):
                    trial_file = f'Trial_{trial}_D{density}_K{kappa_value}_R{rate_value}_S20.csv'
                    file_path = os.path.join(rate_path, trial_file)

                    if os.path.exists(file_path):
                        df = pd.read_csv(file_path)
                        if 'Spawning Time' in df.columns and 'Waiting Time' in df.columns and 'Riding Status' in df.columns:
                            df = df[df['Spawning Time'] > 6999]

                            for status, group in df.groupby('Riding Status'):
                                entries = group['Waiting Time'].dropna().values
                                records = [{
                                    "Case": label,
                                    "Waiting Time": wt,
                                    "Kappa": kappa_value,
                                    "Rate": rate_value,
                                    "Status": status
                                } for wt in entries]

                                if status == "Waiting":
                                    data_still_waiting.extend(records)
                                else:
                                    data_completed.extend(records)

    df_completed = pd.DataFrame(data_completed)
    df_waiting = pd.DataFrame(data_still_waiting)
    return df_completed, df_waiting, sorted(all_kappas), sorted(all_rates)

def plot_combined_boxplots_stacked(df_completed, df_waiting, case_labels, kappa_value, rate_value):
    plt.figure(figsize=(22, 12))
    sns.set_theme(style="ticks", context="talk", font_scale=1.2)

    df_c = df_completed[(df_completed['Kappa'] == kappa_value) & (df_completed['Rate'] == rate_value)].copy()
    df_w = df_waiting[(df_waiting['Kappa'] == kappa_value) & (df_waiting['Rate'] == rate_value)].copy()

    if df_c.empty and df_w.empty:
        print(f"Skipping: No data for Kappa {kappa_value}, Rate {rate_value}")
        return

    df_c['Type'] = 'Completed'
    df_w['Type'] = 'Still Waiting'

    df_combined = pd.concat([df_c, df_w], ignore_index=True)

    # Plotting
    palette = {
        'Completed': '#aec7e8',      # Light blue
        'Still Waiting': '#ffb84d'   # Lighter orange
    }

    sns.boxplot(
        data=df_combined,
        x="Waiting Time",
        y="Case",
        hue="Type",
        palette=palette,
        dodge=True,
        showfliers=False,
        linewidth=1.5,
        orient="h"
    )

    plt.xticks(fontsize=50)
    plt.yticks(fontsize=85)
    plt.xlabel("$t_{wait}$", fontsize=50)
    plt.ylabel("")
    plt.grid(True, linestyle='--', alpha=0.5)
    sns.despine()
    for spine in plt.gca().spines.values():
        spine.set_visible(True)
    plt.legend(fontsize=30, loc='upper right', bbox_to_anchor=(1, 1), ncol=1,frameon=True)
    plt.tight_layout()

    outname = f"Waiting_Times_Stacked_Kappa_{kappa_value}_Rate_{rate_value}_targetted_0.2.png"
    plt.savefig(outname, dpi=200)
    plt.show()
    plt.close()
    print(f"✅ Saved: {outname}")

# ============================
# 🔧 Set your parameters below
# ============================
base_dir = '/home/jovyan/Ruiz_BSThesis'
case_labels = ["OO", r"$O\tilde{T}$", r"$\tilde{J}O$", r"$\tilde{J}\tilde{T}$"]

target_densities = {
    ("OO", "0"): 0.2,
    ("OO", "0.2"): 0.2,
    ("OO", "0.4"): 0.2,
    (r"$O\tilde{T}$", "0"): 0.2,
    (r"$O\tilde{T}$", "0.2"): 0.2,
    (r"$O\tilde{T}$", "0.4"): 0.2,
    (r"$\tilde{J}O$", "0"): 0.2,
    (r"$\tilde{J}O$", "0.2"): 0.2,
    (r"$\tilde{J}O$", "0.4"): 0.2,
    (r"$\tilde{J}\tilde{T}$", "0"): 0.2,
    (r"$\tilde{J}\tilde{T}$", "0.2"): 0.2,
    (r"$\tilde{J}\tilde{T}$", "0.4"): 0.2,
}

# ✅ Extract DataFrames
df_completed, df_waiting, all_kappas, all_rates = extract_waiting_times_per_case(
    base_dir, target_densities, case_labels
)

# 🔁 Plot stacked horizontal boxplots with legend
for kappa in all_kappas:
    for rate in all_rates:
        plot_combined_boxplots_stacked(df_completed, df_waiting, case_labels, kappa, rate)